In [ ]:
import math
import numpy as np
import pandas as pd
import scipy.optimize as opt
import plotly as plt
import plotly.express as pe

from spinorama.filter_iir import Biquad
from spinorama.filter_peq import peq_build, peq_print

In [ ]:
freq = []
target = []
with open("debug_target_02.txt") as fd:
    lines = fd.readlines()
    for line in lines:
        tok = line.split()
        freq.append(float(tok[0]))
        target.append(float(tok[1]))

In [ ]:
plt_target = pe.line(x=freq, y=target, log_x=True)
plt_target

In [ ]:
auto_type = 3
auto_freq = np.array(
    [
        20,
        25,
        31.5,
        40,
        50,
        63,
        80,
        100,
        125,
        160,
        200,
        250,
        315,
        400,
        500,
        630,
        800,
        1000,
        1250,
        1600,
        2000,
        2500,
        3150,
        4000,
        5000,
        6300,
        8000,
        10000,
        12000,
        16000,
        20000,
    ]
)
auto_q = math.sqrt(math.pow(2, 1 / 3)) / (math.pow(2, 1 / 3) - 1)
auto_db = np.zeros(len(auto_freq))
auto_peq = [(1.0, Biquad(auto_type, float(f), 48000, auto_q, float(db))) for f, db in zip(auto_freq, auto_db)]

In [ ]:
def guess(param=1.0, shift=None):
    guess_db = []
    for i, f in enumerate(auto_freq):
        if f < freq[0] or f > freq[-1]:
            db = 0.0
        else:
            db = np.interp(f, freq, target) * param
            if shift is not None:
                db += shift[i][1]
            # 0.25
            db = round(db * 2) / 2
        guess_db.append(db)
    return [(1.0, Biquad(auto_type, float(f), 48000, auto_q, float(db))) for f, db in zip(auto_freq, guess_db)]


peq_print(guess(1))

In [ ]:
def compute_delta(param, shift):
    current_peq = guess(param, shift)
    peq_values = peq_build(auto_freq, current_peq)
    peq_expend = [np.interp(f, auto_freq, peq_values) for f in freq]
    delta = np.array(peq_expend) - np.array(target)
    return delta


def compute_error(param, shift=None):
    delta = compute_delta(param, shift)
    error = np.linalg.norm(delta)
    return error


params = np.linspace(0.1, 1.4, 100)
errors = [compute_error(p, None) for p in params]
pe.line(x=params, y=errors)

In [ ]:
res = opt.minimize(
    fun=lambda x: compute_error(x[0]),
    x0=0.2,
    bounds=[(0.1, 1.4)],
    method="Powell",
)

opt_param = res.x[0]

auto_eq = guess(opt_param)
delta = compute_delta(opt_param, None)
plt = pe.line(x=freq, y=delta, log_x=True)
plt.add_scatter(x=freq, y=peq_build(np.array(freq), auto_eq))
plt.add_scatter(x=freq, y=target)
plt

In [ ]:
plt = pe.line(x=freq, y=target, log_x=True)

for _, peq in auto_eq:
    plt.add_scatter(x=freq, y=peq_build(np.array(freq), [(1.0, peq)]))

plt

In [ ]:
min_error = 1000
min_f = 0
min_p = 0
errors = []
params = np.linspace(-6, 6, 12 * 4 + 1)
shift = [(0, 0) for i in range(len(auto_freq))]
best_shift = []
for i, f in enumerate(auto_freq):
    add = False
    for p in params:
        shift[i] = (f, p)
        error = compute_error(p, shift)
        if error < min_error:
            min_error = error
            min_f = f
            min_p = p
            add = True
    if add:
        best_shift.append((min_f, min_p))
        print("best {} at f={}hz spl={}dB".format(min_error, min_f, min_p))
    else:
        best_shift.append((f, 0))

best_shift

In [ ]:
opt_param = 0.65
auto_eq_prev = guess(opt_param, None)
auto_eq_optim = guess(opt_param, best_shift)
delta = compute_delta(opt_param, best_shift)
print(np.linalg.norm(delta))
plt = pe.line(x=freq, y=delta, log_x=True)
plt.add_scatter(x=freq, y=peq_build(np.array(freq), auto_eq_prev))
plt.add_scatter(x=freq, y=peq_build(np.array(freq), auto_eq_optim))
plt.add_scatter(x=freq, y=target)
plt

In [ ]:
peq_print(auto_eq_optim)

In [ ]:
target